In [7]:
import numpy as np
import pandas as pd

In [8]:
#print('输出A文件路径',inputfile=input())

In [9]:
"""
读取txt文件
该文本中的分割符既有空格又有制表符（‘/t’），sep参数用‘/s+’，可以匹配任何空格。
"""
#header=None:没有每列的column name，可以自己设定
#encoding='gb2312':其他编码中文显示错误
#sep=',':用逗号来分隔每行的数据
#index_col=0:设置第1列数据作为index
#index_col=0  指定第一列作为索引
inputfile='A53859-201912.TXT'

In [10]:
data = pd.read_csv(inputfile,header=None,sep='/t',encoding='gb2312',index_col=0)

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [11]:
#提取数据
SJ=list(data.index)
#print(SJ)

In [12]:
#SJ[0]为站点相关信息，进行提取
station=(SJ[0]).split(' ',-1)
jiben=['区站号','纬度','经度','观测场海波高度', '气压传感器海拔高度', '风速感应器距地（平台）高度',
         '观测平台距地高度','观测方式和测站类别', '观测项目标识' ,'质量控制指示码', '年份' ,'月份']
print('台站参数：')
df_station=pd.DataFrame(station,index=jiben,columns=['数据'])
print(df_station)

台站参数：
                                 数据
区站号                           53859
纬度                            3605N
经度                           11040E
观测场海波高度                      008513
气压传感器海拔高度                    008523
风速感应器距地（平台）高度                   102
观测平台距地高度                        000
观测方式和测站类别                       S12
观测项目标识         11111099111009111011
质量控制指示码                           1
年份                             2019
月份                               12


In [13]:
'''
观测数据
每天4次定时观测时间分别为02、08、14、20时；
每天3次定时观测时间分别为08、14、20时；
每天24次定时观测时间分别为21时至20时，每1小时观测一次。
'''

'\n观测数据\n每天4次定时观测时间分别为02、08、14、20时；\n每天3次定时观测时间分别为08、14、20时；\n每天24次定时观测时间分别为21时至20时，每1小时观测一次。\n'

In [14]:
'''
指示码：
P：本站气压
方位码（X）：
C：X=C 
    PC_1:本站气压每天24次定时值和日最高、最低值及出现时间共28组,分为两个记录
            第一个记录（21-08时）为12组，第二个记录（09-20时和最高及出现时间、最低值及出现时间）为16组
    PC_2:海平面气压每天4次定时值共4组。
单位：0.1hpa
'''
PC=SJ[SJ.index('PC'):SJ.index('TB')]
print(len(PC))
#通过等号，将数据划分成两部分
pc=[]
for i in range(len(PC)):
    if PC[i][-1]=='=':  # 若字符串最后一位为=
        pc=pc+[i]
print(pc)

94
[62, 93]


In [15]:
#PC_1 第一组
PC_1=PC[1:pc[0]+1]
print(len(PC_1))
#print(PC_1)

62


In [16]:
#对 PC_1 内的数据进行拼接,生成pc_1
pc_1=[]  
for i in range(len(PC_1)+1):
    if i != 0:
        if i%2==0:   #能被2整除
            pc_1=pc_1+[PC_1[i-2]+' '+PC_1[i-1]]
print(len(pc_1))
#print(pc_1)

31


In [17]:
#去掉.和对不同的=结尾进行分割,在 pc_1 上直接进行处理
pc0=['=','.']
for i in range(len(pc_1)):
    if pc_1[i][-1] in pc0:
        pc_1[i]=pc_1[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        pc_1[i]=pc_1[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(pc_1)):
    pc_1[i]=pc_1[i].split()
print(len(pc_1))
#print(pc_1)

31


In [18]:
#格式转换
for i in range(len(pc_1)):
    for j in range(len(pc_1[i])):
        if j == 25 or j==27:
            pc_1[i][j]='%.2f' %(int(pc_1[i][j])/100)
            pc_1[i][j]=pc_1[i][j].replace('.',':')
        else:
            pc_1[i][j]='%.2f' %(int(pc_1[i][j])/10)

In [19]:
year=df_station.loc['年份'].values
month=df_station.loc['月份'].values
#print(month)
index=[]
for i in range(1,len(pc_1)+1):
    index=index+["{}年-{}月-{:0>2d}日".format(year[0],month[0],i)]
#print(index)

In [20]:
#新建dataframe
hour_28=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时',
         '最大','时间','最小','时间',]
print('本站气压：')
df_PC_1=pd.DataFrame(data=pc_1,index=index,columns=hour_28)
print(df_PC_1)

本站气压：
                  21时     22时     23时     24时      1时      2时      3时      4时  \
2019年-12月-01日  930.20  930.70  931.00  930.70  930.50  930.60  930.40  930.10   
2019年-12月-02日  931.50  932.20  932.80  933.10  933.00  932.90  933.00  932.90   
2019年-12月-03日  930.40  930.50  930.50  930.20  929.90  929.70  929.70  929.20   
2019年-12月-04日  931.00  931.50  931.90  932.00  932.00  932.00  932.50  932.50   
2019年-12月-05日  934.40  934.60  934.80  934.80  934.50  934.60  934.80  934.80   
2019年-12月-06日  935.80  935.80  935.80  935.70  935.40  935.40  935.20  934.50   
2019年-12月-07日  930.70  930.60  930.70  930.50  930.20  930.10  930.30  930.10   
2019年-12月-08日  929.30  929.30  929.10  928.70  928.40  928.00  927.90  927.50   
2019年-12月-09日  925.00  925.00  924.90  924.90  924.60  924.40  924.20  923.50   
2019年-12月-10日  921.00  921.10  921.20  920.90  920.70  920.60  920.60  921.10   
2019年-12月-11日  923.80  924.10  924.20  924.40  924.40  924.40  924.50  924.30   
2019年-12月-12日  926.00 

In [21]:
#PC_2 第二组
PC_2=PC[pc[0]+1:pc[1]+1]
print(len(PC_2))
#print(PC_2)

31


In [22]:
#去掉.和对不同的=结尾进行分割
for i in range(len(PC_2)):
    if PC_2[i][-1] in pc0:
        PC_2[i]=PC_2[i].rstrip('.')  #去除.
        PC_2[i]=PC_2[i].rstrip('=')  #去除=

In [23]:
#进行分割
for i in range(len(PC_2)):
    PC_2[i]=PC_2[i].split()
print(len(PC_2))
#print(PC_2)

31


In [24]:
#新建dataframe
hour_4=['00时','06时','12时','18时', ]
print('海平面气压：')
df_PC_2=pd.DataFrame(data=PC_2,index=index,columns=hour_4)
print(df_PC_2)

海平面气压：
                00时   06时   12时   18时
2019年-12月-01日  0347  0358  0317  0361
2019年-12月-02日  0366  0405  0350  0365
2019年-12月-03日  0346  0365  0317  0366
2019年-12月-04日  0360  0405  0361  0402
2019年-12月-05日  0387  0427  0387  0422
2019年-12月-06日  0397  0405  0340  0362
2019年-12月-07日  0338  0365  0307  0343
2019年-12月-08日  0307  0324  0266  0290
2019年-12月-09日  0264  0282  0211  0243
2019年-12月-10日  0214  0255  0240  0269
2019年-12月-11日  0268  0307  0269  0305
2019年-12月-12日  0283  0297  0270  0269
2019年-12月-13日  0272  0291  0277  0312
2019年-12月-14日  0293  0312  0272  0258
2019年-12月-15日  0242  0240  0219  0234
2019年-12月-16日  0244  0238  0224  0271
2019年-12月-17日  0286  0324  0301  0339
2019年-12月-18日  0337  0354  0298  0329
2019年-12月-19日  0321  0346  0303  0341
2019年-12月-20日  0304  0309  0248  0294
2019年-12月-21日  0272  0285  0220  0249
2019年-12月-22日  0218  0232  0174  0211
2019年-12月-23日  0214  0266  0250  0281
2019年-12月-24日  0255  0255  0225  0234
2019年-12月-25日  0224  0242  0250  0324
2019年

In [25]:
'''
T:气温
B:每天24次定时和日最高、最低值及出现时间共28组，分为2个记录，
    第1个记录（21～08时）为12组，第2个记录（09～20时和最高值及出现时间、最低值及出现时间）为16组。
单位：0.1摄氏度
'''
TB=SJ[SJ.index('TB'):SJ.index('IB')]
#print(TB)
print(len(TB))
print(TB[-1])

63
-114 -082 -051 -030 -027 -024 -026 -029 -026 -025 -026 -026 -022 1415 -130 0738=


In [26]:
#对数据进行拼接
tb1=[]  
for i in range(len(TB)+1):
    if i != 0:
        if i%2==0:   #能被2整除
            tb1=tb1+[TB[i-1]+' '+TB[i]]
#print(tb1[0][-1])
print(len(tb1))
#print(tb1)

31


In [27]:
#去掉.和对不同的=结尾进行分割
tb0=['=','.']
tb2=[]
for i in range(len(tb1)):
    if tb1[i][-1] in pc0:
        tb1[i].rstrip('.')
        tb2=tb2+[tb1[i][:-1]]
for i in range(len(tb2)):
    tb2[i]=tb2[i].split()
print(len(tb2))

31


In [28]:
#格式转换
for i in range(len(tb2)):
    for j in range(len(tb2[i])):
        if j == 25 or j==27:   #时间格式处理
            tb2[i][j]='%.2f' %(int(tb2[i][j])/100)
            tb2[i][j]=tb2[i][j].replace('.',':')
        else:
            tb2[i][j]=float('%.2f' %(float(tb2[i][j])/10))

In [29]:
#新建dataframe
print('气温：')
df_TB=pd.DataFrame(data=tb2,index=index,columns=hour_28)
print(df_TB)

气温：
               21时  22时  23时  24时   1时    2时    3时    4时    5时    6时  ...  \
2019年-12月-01日 -0.2 -0.3 -0.7 -1.2 -1.7  -2.3  -2.6  -2.4  -2.8  -3.1  ...   
2019年-12月-02日  1.6  1.0 -0.3 -1.4 -2.4  -3.6  -4.5  -6.2  -7.1  -7.9  ...   
2019年-12月-03日 -4.2 -5.1 -5.9 -6.3 -7.6  -7.7  -7.9  -8.3  -9.1  -9.2  ...   
2019年-12月-04日 -2.7 -3.7 -4.3 -5.2 -5.9  -6.4  -6.7  -7.6  -8.1  -8.1  ...   
2019年-12月-05日 -2.2 -3.3 -3.9 -4.8 -5.4  -6.1  -6.5  -7.0  -7.6  -8.1  ...   
2019年-12月-06日 -2.3 -3.1 -3.8 -4.4 -4.9  -5.6  -6.1  -6.3  -7.0  -7.6  ...   
2019年-12月-07日 -2.4 -3.1 -4.4 -4.5 -4.9  -5.8  -6.2  -6.7  -6.6  -7.4  ...   
2019年-12月-08日 -1.0 -1.6 -2.1 -2.7 -3.1  -3.6  -4.0  -4.5  -4.8  -5.0  ...   
2019年-12月-09日 -0.3 -1.1 -1.9 -2.3 -3.0  -3.2  -3.4  -3.9  -5.0  -4.9  ...   
2019年-12月-10日  1.0 -0.2 -1.1 -1.6 -2.4  -3.0  -3.1  -3.7  -3.7  -3.7  ...   
2019年-12月-11日 -1.0 -2.0 -2.7 -3.2 -4.1  -4.9  -5.8  -5.6  -6.6  -6.8  ...   
2019年-12月-12日 -1.5 -1.9 -2.7 -3.3 -4.3  -4.8  -4.9  -5.6  -6.1  -6.2  ..

In [30]:
df_TB['总和']=df_TB['21时']+df_TB['22时']+df_TB['23时']+df_TB['24时']+df_TB['1时']+df_TB['2时']+\
            df_TB['3时']+df_TB['4时']+df_TB['5时']+df_TB['6时']+df_TB['7时']+df_TB['8时']+\
            df_TB['9时']+df_TB['10时']+df_TB['11时']+df_TB['12时']+df_TB['13时']+df_TB['14时']+\
            df_TB['15时']+df_TB['16时']+df_TB['17时']+df_TB['18时']+df_TB['19时']+df_TB['20时']
print(df_TB['总和'])

2019年-12月-01日      0.5
2019年-12月-02日    -50.2
2019年-12月-03日    -64.3
2019年-12月-04日    -41.7
2019年-12月-05日    -45.2
2019年-12月-06日    -36.7
2019年-12月-07日    -27.5
2019年-12月-08日     11.5
2019年-12月-09日     48.2
2019年-12月-10日     38.1
2019年-12月-11日    -14.7
2019年-12月-12日    -42.8
2019年-12月-13日     39.9
2019年-12月-14日     16.2
2019年-12月-15日     20.3
2019年-12月-16日     40.9
2019年-12月-17日     15.9
2019年-12月-18日      6.9
2019年-12月-19日    -59.5
2019年-12月-20日    -45.5
2019年-12月-21日    -24.3
2019年-12月-22日    -27.2
2019年-12月-23日    -59.8
2019年-12月-24日     20.1
2019年-12月-25日     29.6
2019年-12月-26日    -63.0
2019年-12月-27日    -50.7
2019年-12月-28日     -3.6
2019年-12月-29日     -1.1
2019年-12月-30日    -77.4
2019年-12月-31日   -175.4
Name: 总和, dtype: float64


In [31]:
df_TB['平均']=round(df_TB['总和']/24,1)
print(df_TB['平均'])

2019年-12月-01日    0.0
2019年-12月-02日   -2.1
2019年-12月-03日   -2.7
2019年-12月-04日   -1.7
2019年-12月-05日   -1.9
2019年-12月-06日   -1.5
2019年-12月-07日   -1.1
2019年-12月-08日    0.5
2019年-12月-09日    2.0
2019年-12月-10日    1.6
2019年-12月-11日   -0.6
2019年-12月-12日   -1.8
2019年-12月-13日    1.7
2019年-12月-14日    0.7
2019年-12月-15日    0.8
2019年-12月-16日    1.7
2019年-12月-17日    0.7
2019年-12月-18日    0.3
2019年-12月-19日   -2.5
2019年-12月-20日   -1.9
2019年-12月-21日   -1.0
2019年-12月-22日   -1.1
2019年-12月-23日   -2.5
2019年-12月-24日    0.8
2019年-12月-25日    1.2
2019年-12月-26日   -2.6
2019年-12月-27日   -2.1
2019年-12月-28日   -0.2
2019年-12月-29日   -0.0
2019年-12月-30日   -3.2
2019年-12月-31日   -7.3
Name: 平均, dtype: float64


In [32]:
print(df_TB.columns)
print("气温数据处理完毕")

Index(['21时', '22时', '23时', '24时', '1时', '2时', '3时', '4时', '5时', '6时', '7时',
       '8时', '9时', '10时', '11时', '12时', '13时', '14时', '15时', '16时', '17时',
       '18时', '19时', '20时', '最大', '时间', '最小', '时间', '总和', '平均'],
      dtype='object')
气温数据处理完毕


In [33]:
'''
U:相对湿度
B:每天24次定时值和自动观测日最小值及出现时间共26组，分为2个记录，
    第1个记录（21～08时）为12组，
    第2个记录（09～20时和最小值及出现时间）为14组。 
'''
UB=SJ[SJ.index('UB'):SJ.index('N9')]
#print(UB)
print(len(UB))
#print(UB[-1])

63


In [34]:
#对数据进行拼接
ub1=[]  
for i in range(len(UB)+1):
    if i != 0:
        if i%2==0:   #能被2整除
            ub1=ub1+[UB[i-1]+' '+UB[i]]
#print(tb1[0][-1])
print(len(ub1))
#print(ub1)

31


In [35]:
#去掉.和对不同的=结尾进行分割
ub0=['=','.']
ub2=[]
for i in range(len(ub1)):
    if ub1[i][-1] in ub0:
        ub1[i].rstrip('.')
        ub2=ub2+[ub1[i][:-1]]
#print(a2)
for i in range(len(ub2)):
    ub2[i]=ub2[i].split()
#print(ub2)
print(len(ub2))

31


In [36]:
#格式转换
for i in range(len(ub2)):
    for j in range(len(ub2[i])):
        if j == 25:
            ub2[i][j]='%.2f' %(int(ub2[i][j])/100)
            ub2[i][j]=ub2[i][j].replace('.',':')
        else:
            if ub2[i][j]=='%%':
                ub2[i][j]=int(100)
            else:
                ub2[i][j]=int(ub2[i][j])

In [37]:
#新建dataframe
print('相对湿度：')
hour_26=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时',
         '最小湿度','时间',]
df_UB=pd.DataFrame(data=ub2,index=index,columns=hour_26)
print(df_UB)

相对湿度：
               21时  22时  23时  24时  1时  2时  3时  4时  5时  6时  ...  13时  14时  15时  \
2019年-12月-01日   63   68   73   76  74  82  72  66  64  67  ...   35   27   26   
2019年-12月-02日   24   24   28   29  31  35  41  51  57  61  ...   22   22   20   
2019年-12月-03日   47   50   52   54  63  61  62  65  69  70  ...   24   14   13   
2019年-12月-04日   44   48   50   54  57  60  61  65  66  68  ...   22   18   18   
2019年-12月-05日   47   51   53   56  60  62  64  67  69  71  ...   30   26   22   
2019年-12月-06日   48   51   53   55  58  62  64  64  67  70  ...   35   31   28   
2019年-12月-07日   58   59   66   64  65  70  72  74  73  77  ...   34   31   31   
2019年-12月-08日   65   68   70   73  75  77  80  83  86  89  ...   31   22   20   
2019年-12月-09日   51   54   57   59  64  65  66  67  72  71  ...   19   18   16   
2019年-12月-10日   48   53   56   58  62  64  64  66  66  65  ...   22   19   18   
2019年-12月-11日   46   49   51   52  56  59  63  62  67  69  ...   28   24   21   
2019年-12月-12日   47   4

In [38]:
df_UB['相对湿度总和']=df_UB['21时']+df_UB['22时']+df_UB['23时']+df_UB['24时']+df_UB['1时']+df_UB['2时']+\
            df_UB['3时']+df_UB['4时']+df_UB['5时']+df_UB['6时']+df_UB['7时']+df_UB['8时']+\
            df_UB['9时']+df_UB['10时']+df_UB['11时']+df_UB['12时']+df_UB['13时']+df_UB['14时']+\
            df_UB['15时']+df_UB['16时']+df_UB['17时']+df_UB['18时']+df_UB['19时']+df_UB['20时']
print(df_UB['相对湿度总和'])

2019年-12月-01日    1356
2019年-12月-02日     880
2019年-12月-03日    1094
2019年-12月-04日    1084
2019年-12月-05日    1189
2019年-12月-06日    1230
2019年-12月-07日    1387
2019年-12月-08日    1452
2019年-12月-09日    1158
2019年-12月-10日    1066
2019年-12月-11日    1128
2019年-12月-12日    1660
2019年-12月-13日    1907
2019年-12月-14日    1575
2019年-12月-15日    1632
2019年-12月-16日    2021
2019年-12月-17日    1459
2019年-12月-18日    1334
2019年-12月-19日    1489
2019年-12月-20日    1471
2019年-12月-21日    1625
2019年-12月-22日    1532
2019年-12月-23日    1550
2019年-12月-24日    1755
2019年-12月-25日    1628
2019年-12月-26日    1053
2019年-12月-27日     987
2019年-12月-28日     956
2019年-12月-29日    1081
2019年-12月-30日     759
2019年-12月-31日     731
Name: 相对湿度总和, dtype: int64


In [39]:
df_UB['平均相对湿度']=round(df_UB['相对湿度总和']/24,0)
print(df_UB['平均相对湿度'])

2019年-12月-01日    56.0
2019年-12月-02日    37.0
2019年-12月-03日    46.0
2019年-12月-04日    45.0
2019年-12月-05日    50.0
2019年-12月-06日    51.0
2019年-12月-07日    58.0
2019年-12月-08日    60.0
2019年-12月-09日    48.0
2019年-12月-10日    44.0
2019年-12月-11日    47.0
2019年-12月-12日    69.0
2019年-12月-13日    79.0
2019年-12月-14日    66.0
2019年-12月-15日    68.0
2019年-12月-16日    84.0
2019年-12月-17日    61.0
2019年-12月-18日    56.0
2019年-12月-19日    62.0
2019年-12月-20日    61.0
2019年-12月-21日    68.0
2019年-12月-22日    64.0
2019年-12月-23日    65.0
2019年-12月-24日    73.0
2019年-12月-25日    68.0
2019年-12月-26日    44.0
2019年-12月-27日    41.0
2019年-12月-28日    40.0
2019年-12月-29日    45.0
2019年-12月-30日    32.0
2019年-12月-31日    30.0
Name: 平均相对湿度, dtype: float64


In [40]:
print(df_UB.columns)
print('相对湿度处理完毕')

Index(['21时', '22时', '23时', '24时', '1时', '2时', '3时', '4时', '5时', '6时', '7时',
       '8时', '9时', '10时', '11时', '12时', '13时', '14时', '15时', '16时', '17时',
       '18时', '19时', '20时', '最小湿度', '时间', '相对湿度总和', '平均相对湿度'],
      dtype='object')
相对湿度处理完毕


In [41]:
'''
R:降水量
6:X=6。
    定时降水量段每天3组（20-8）、（8-20）、（20-20）；
    自记降水量段每天（21-20时）共24组，分为2个记录，每个记录为12组；
    降水上下连接值段每月3组。
    无降水量-0000，微量-,,,,
'''
R6=SJ[SJ.index('R6'):SJ.index('W0')]
#print(R6)
print(len(R6))

95


In [42]:
#通过等号，将数据划分成三部分
r6=[]
for i in range(len(R6)):
    if R6[i][-1]=='=':  # 若字符串最后一位为=
        r6=r6+[i]
print(r6)

[31, 93, 94]


In [43]:
#R6_1 定时降水量
R6_1=R6[1:r6[0]+1]
print(len(R6_1))
print(R6_1)

31
['0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0004 0004', ',,,, 0000 ,,,,', ',,,, 0002 0002', '0000 0001 0001', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000=']


In [44]:
#去掉.和对不同的=结尾进行分割,在r61上直接进行处理
r60=['=','.']
for i in range(len(R6_1)):
    if R6_1[i][-1] in r60:
        R6_1[i]=R6_1[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        R6_1[i]=R6_1[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(R6_1)):
    R6_1[i]=R6_1[i].split()
#print(R6_1)
print(len(R6_1))

31


In [45]:
#格式转换
for i in range(len(R6_1)):
    for j in range(len(R6_1[i])):
        if R6_1[i][j]!= ',,,,':
            R6_1[i][j]='%.1f' %(int(R6_1[i][j])/10)
        else:
            R6_1[i][j].replace(',,,,','0.0')

In [46]:
#新建dataframe
print('定时降水量：')
hour_3=['20-8时','8-20时','20-20时']
df_R6_1=pd.DataFrame(data=R6_1,index=index,columns=hour_3)
print(df_R6_1)

定时降水量：
              20-8时 8-20时 20-20时
2019年-12月-01日   0.0   0.0    0.0
2019年-12月-02日   0.0   0.0    0.0
2019年-12月-03日   0.0   0.0    0.0
2019年-12月-04日   0.0   0.0    0.0
2019年-12月-05日   0.0   0.0    0.0
2019年-12月-06日   0.0   0.0    0.0
2019年-12月-07日   0.0   0.0    0.0
2019年-12月-08日   0.0   0.0    0.0
2019年-12月-09日   0.0   0.0    0.0
2019年-12月-10日   0.0   0.0    0.0
2019年-12月-11日   0.0   0.0    0.0
2019年-12月-12日   0.0   0.4    0.4
2019年-12月-13日  ,,,,   0.0   ,,,,
2019年-12月-14日  ,,,,   0.2    0.2
2019年-12月-15日   0.0   0.1    0.1
2019年-12月-16日   0.0   0.0    0.0
2019年-12月-17日   0.0   0.0    0.0
2019年-12月-18日   0.0   0.0    0.0
2019年-12月-19日   0.0   0.0    0.0
2019年-12月-20日   0.0   0.0    0.0
2019年-12月-21日   0.0   0.0    0.0
2019年-12月-22日   0.0   0.0    0.0
2019年-12月-23日   0.0   0.0    0.0
2019年-12月-24日   0.0   0.0    0.0
2019年-12月-25日   0.0   0.0    0.0
2019年-12月-26日   0.0   0.0    0.0
2019年-12月-27日   0.0   0.0    0.0
2019年-12月-28日   0.0   0.0    0.0
2019年-12月-29日   0.0   0.0    0.0
201

In [47]:
r62=R6[r6[0]+1:r6[1]+1]
print(len(r62))
#print(r62)

62


In [48]:
#对数据进行拼接
R6_2=[]
for i in range(len(r62)+1):
    if i != 0:
        if i%2==0:   #能被2整除
            R6_2=R6_2+[r62[i-2]+' '+r62[i-1]]
print(len(R6_2))
#print(R6_2)

31


In [49]:
#去掉.和对不同的=结尾进行分割
for i in range(len(R6_2)):
    if R6_2[i][-1] in r60:
        R6_2[i]=R6_2[i].rstrip('.')  #去除.
        R6_2[i]=R6_2[i].rstrip('=')  #去除=
print(len(R6_2))
#print(R6_2)

31


In [50]:
for i in range(len(R6_2)):
    R6_2[i]=R6_2[i].split()
print(len(R6_2))
#print(R6_2)

31


In [51]:
#格式转换
for i in range(len(R6_2)):
    for j in range(len(R6_2[i])):
        if R6_2[i][j]!= ',,,,':
            R6_2[i][j]=float('%.1f' %(int(R6_2[i][j])/10))
        else:
            R6_2[i][j].replace(',,,,','0.0')

In [52]:
#新建dataframe
hour_24=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时']
print('定时降水量：')
df_R6_2=pd.DataFrame(data=R6_2,index=index,columns=hour_24)
print(df_R6_2)

定时降水量：
               21时  22时  23时  24时   1时   2时   3时   4时   5时   6时  ...  11时  \
2019年-12月-01日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-12月-02日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-12月-03日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-12月-04日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-12月-05日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-12月-06日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-12月-07日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-12月-08日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-12月-09日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-12月-10日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-12月-11日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-12月-12日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ... 

In [53]:
df_R6_2['总降水量']=df_R6_2['21时']+df_R6_2['22时']+df_R6_2['23时']+df_R6_2['24时']+\
                df_R6_2['1时']+df_R6_2['2时']+df_R6_2['3时']+df_R6_2['4时']+df_R6_2['5时']+df_R6_2['6时']+\
                df_R6_2['7时']+df_R6_2['8时']+df_R6_2['9时']+df_R6_2['10时']+df_R6_2['11时']+\
                df_R6_2['12时']+df_R6_2['13时']+df_R6_2['14时']+df_R6_2['15时']+df_R6_2['16时']+df_R6_2['17时']+\
                df_R6_2['18时']+df_R6_2['19时']+df_R6_2['20时']
print(df_R6_2['总降水量'])

2019年-12月-01日    0.0
2019年-12月-02日    0.0
2019年-12月-03日    0.0
2019年-12月-04日    0.0
2019年-12月-05日    0.0
2019年-12月-06日    0.0
2019年-12月-07日    0.0
2019年-12月-08日    0.0
2019年-12月-09日    0.0
2019年-12月-10日    0.0
2019年-12月-11日    0.0
2019年-12月-12日    0.4
2019年-12月-13日    0.0
2019年-12月-14日    0.2
2019年-12月-15日    0.1
2019年-12月-16日    0.0
2019年-12月-17日    0.0
2019年-12月-18日    0.0
2019年-12月-19日    0.0
2019年-12月-20日    0.0
2019年-12月-21日    0.0
2019年-12月-22日    0.0
2019年-12月-23日    0.0
2019年-12月-24日    0.0
2019年-12月-25日    0.0
2019年-12月-26日    0.0
2019年-12月-27日    0.0
2019年-12月-28日    0.0
2019年-12月-29日    0.0
2019年-12月-30日    0.0
2019年-12月-31日    0.0
Name: 总降水量, dtype: float64


In [54]:
#r63占时不用
#r63=R6[r6[1]+1:r6[2]+1]
#print(len(r63))
#print(r63)

In [55]:
print(df_R6_2.columns)
print("降水量数据处理完毕")

Index(['21时', '22时', '23时', '24时', '1时', '2时', '3时', '4时', '5时', '6时', '7时',
       '8时', '9时', '10时', '11时', '12时', '13时', '14时', '15时', '16时', '17时',
       '18时', '19时', '20时', '总降水量'],
      dtype='object')
降水量数据处理完毕


In [56]:
'''
F:风
N:X=N。
    第1段和第2段，每段每天24次定时值共24组，分为4个记录，每个记录为6组；
    风向风速每组6位，第一段和第二段前3位为风向，后三位为风速，
    最大极大风前3位为风速、后三位为风向。
    第3段每天最大、极大风共4组，第2、4组为出现时间。 
风速单位:0.1m/s
风向单位：度
'''
FN=SJ[SJ.index('FN'):SJ.index('DB')]
print(len(FN))
fn=[]
for i in range(len(FN)):
    if FN[i][-1]=='=':  # 若字符串最后一位为=
        fn=fn+[i]
print(fn)

280
[124, 248, 279]


In [57]:
#FN_1 第一段：两分钟平均风速风向
FN_1=FN[1:fn[0]+1]
print(len(FN_1))
#print(FN_1)

124


In [58]:
#对 FN_1 内的数据进行拼接,生成  fn_1
fn_1=[]  
for i in range(len(FN_1)+1):
    if i != 0:
        if i%4==0:   #能被 4 整除
            fn_1=fn_1+[FN_1[i-4]+' '+FN_1[i-3]+' '+FN_1[i-2]+' '+FN_1[i-1]]
print(len(fn_1))
#print(fn_1)

31


In [59]:
#去掉.和对不同的=结尾进行分割,在 pc_1 上直接进行处理
fn0=['=','.']
for i in range(len(fn_1)):
    if fn_1[i][-1] in pc0:
        fn_1[i]=fn_1[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        fn_1[i]=fn_1[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(fn_1)):
    fn_1[i]=fn_1[i].split()
print(len(fn_1))
#print(fn_1)

31


In [60]:
#格式转换
fx=[]  #风向
fs=[]  #风速
for i in range(len(fn_1)):
    for j in range(len(fn_1[i])):
            fx=fx+[fn_1[i][j][:3]]
            fs=fs+[int(fn_1[i][j][3:])/10]

In [61]:
fx_point=[]  #方位
fx_angle=[]  #中心角度
'''
风向按照度数进行划分，划分为16个风向

'''
for i in range(len(fx)):
    if fx[i]=='PPC':
        fx_point=fx_point+['PPC']    #C-PPC 静风，风速小于0.2m/s 
        fx_angle=fx_angle+[0]
    else:
        if int(fx[i])>=0 and int(fx[i])<=11.25:
            fx_point=fx_point+['N']
            fx_angle=fx_angle+[0]
        elif int(fx[i])>=11.26 and int(fx[i])<=33.75:
            fx_point=fx_point+['NNE']
            fx_angle=fx_angle+[22.5]
        elif int(fx[i])>=33.76 and int(fx[i])<=56.25:
            fx_point=fx_point+['NE']
            fx_angle=fx_angle+[45]
        elif int(fx[i])>=56.26 and int(fx[i])<=78.75:
            fx_point=fx_point+['ENE']
            fx_angle=fx_angle+[67.5]
        elif int(fx[i])>=78.76 and int(fx[i])<=101.25:
            fx_point=fx_point+['E']
            fx_angle=fx_angle+[90]
        elif int(fx[i])>=101.26 and int(fx[i])<=123.75:
            fx_point=fx_point+['ESE']
            fx_angle=fx_angle+[112.5]
        if int(fx[i])>=123.76 and int(fx[i])<=146.25:
            fx_point=fx_point+['SE']
            fx_angle=fx_angle+[135]
        elif int(fx[i])>=146.26 and int(fx[i])<=168.75:
            fx_point=fx_point+['SSE']
            fx_angle=fx_angle+[157.5]
        elif int(fx[i])>=168.76 and int(fx[i])<=191.25:
            fx_point=fx_point+['S']
            fx_angle=fx_angle+[180]
        elif int(fx[i])>=191.26 and int(fx[i])<=213.75:
            fx_point=fx_point+['SSW']
            fx_angle=fx_angle+[202.5]
        elif int(fx[i])>=213.76 and int(fx[i])<=236.25:
            fx_point=fx_point+['SW']
            fx_angle=fx_angle+[225]
        elif int(fx[i])>=236.26 and int(fx[i])<=258.75:
            fx_point=fx_point+['WSW']
            fx_angle=fx_angle+[247.5]
        elif int(fx[i])>=258.76 and int(fx[i])<=281.25:
            fx_point=fx_point+['W']
            fx_angle=fx_angle+[270]
        elif int(fx[i])>=281.76 and int(fx[i])<=303.75:
            fx_point=fx_point+['WNW']
            fx_angle=fx_angle+[295.5]
        elif int(fx[i])>=303.76 and int(fx[i])<=326.25:
            fx_point=fx_point+['NW']
            fx_angle=fx_angle+[315]
        elif int(fx[i])>=326.26 and int(fx[i])<=348.75:
            fx_point=fx_point+['NNW']
            fx_angle=fx_angle+[337.5]
        elif int(fx[i])>=348.76 and int(fx[i])<=360:
            fx_point=fx_point+['N']
            fx_angle=fx_angle+[0]

In [62]:
fx1_point=np.array(fx_point).reshape(-1,24) 
print(len(fx1_point))  #方位

31


In [63]:
fx1_angle=np.array(fx_angle).reshape(-1,24) 
print(len(fx1_angle))

31


In [64]:
fs1=np.array(fs).reshape(-1,24) 
print(len(fs1))

31


In [65]:
#新建dataframe
hour_24=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时']
print('2分钟平均风速：')
df_FN1_fs=pd.DataFrame(data=fs1,index=index,columns=hour_24)
print(df_FN1_fs)

2分钟平均风速：
               21时  22时  23时  24时   1时   2时   3时   4时   5时   6时  ...  11时  \
2019年-12月-01日  0.0  0.0  0.0  0.9  0.0  0.7  1.4  0.0  0.8  0.3  ...  3.7   
2019年-12月-02日  2.6  4.5  2.4  2.3  2.4  1.0  1.5  1.6  0.0  0.0  ...  3.1   
2019年-12月-03日  3.1  3.1  3.7  1.8  1.7  2.2  2.5  2.0  1.3  1.4  ...  2.3   
2019年-12月-04日  2.0  2.4  3.4  1.9  1.7  1.7  1.8  1.7  0.9  1.8  ...  2.7   
2019年-12月-05日  2.4  2.6  2.8  1.5  1.8  2.0  1.3  1.6  1.3  1.4  ...  2.2   
2019年-12月-06日  2.8  2.6  3.0  2.8  2.1  2.2  2.2  2.5  1.5  1.1  ...  1.4   
2019年-12月-07日  1.9  2.3  1.6  3.3  2.7  1.4  1.4  1.4  1.8  0.9  ...  1.6   
2019年-12月-08日  1.6  2.0  1.8  2.2  2.0  2.4  1.6  1.9  1.9  2.9  ...  3.3   
2019年-12月-09日  1.9  2.8  1.9  2.3  2.4  2.7  2.4  1.7  1.1  2.2  ...  1.2   
2019年-12月-10日  2.5  2.1  2.2  2.6  1.8  2.0  1.9  1.0  0.8  1.5  ...  1.1   
2019年-12月-11日  2.9  2.2  2.2  2.9  2.1  1.3  1.4  2.2  1.1  1.4  ...  2.9   
2019年-12月-12日  2.3  1.9  2.9  2.4  1.5  1.9  2.1  1.0  0.9  1.3  ..

In [66]:
df_FN1_fs['2分钟定时风速']=df_FN1_fs['21时']+df_FN1_fs['22时']+df_FN1_fs['23时']+df_FN1_fs['24时']+\
            df_FN1_fs['1时']+df_FN1_fs['2时']+df_FN1_fs['3时']+df_FN1_fs['4时']+df_FN1_fs['5时']+\
            df_FN1_fs['6时']+df_FN1_fs['7时']+df_FN1_fs['8时']+df_FN1_fs['9时']+df_FN1_fs['10时']+\
            df_FN1_fs['11时']+df_FN1_fs['12时']+df_FN1_fs['13时']+df_FN1_fs['14时']+df_FN1_fs['15时']+\
            df_FN1_fs['16时']+df_FN1_fs['17时']+df_FN1_fs['18时']+df_FN1_fs['19时']+df_FN1_fs['20时']
print(df_FN1_fs['2分钟定时风速'])

2019年-12月-01日    27.1
2019年-12月-02日    44.2
2019年-12月-03日    48.4
2019年-12月-04日    48.1
2019年-12月-05日    46.4
2019年-12月-06日    50.3
2019年-12月-07日    40.8
2019年-12月-08日    54.4
2019年-12月-09日    47.1
2019年-12月-10日    55.3
2019年-12月-11日    46.6
2019年-12月-12日    41.9
2019年-12月-13日    32.5
2019年-12月-14日    40.4
2019年-12月-15日    33.5
2019年-12月-16日    41.0
2019年-12月-17日    43.6
2019年-12月-18日    33.3
2019年-12月-19日    49.0
2019年-12月-20日    56.0
2019年-12月-21日    46.3
2019年-12月-22日    51.8
2019年-12月-23日    38.2
2019年-12月-24日    27.9
2019年-12月-25日    56.2
2019年-12月-26日    54.4
2019年-12月-27日    57.9
2019年-12月-28日    65.0
2019年-12月-29日    61.6
2019年-12月-30日    57.6
2019年-12月-31日    49.6
Name: 2分钟定时风速, dtype: float64


In [67]:
df_FN1_fs['2分钟定时风速平均']=round(df_FN1_fs['2分钟定时风速']/24,1)
print(df_FN1_fs['2分钟定时风速平均'])

2019年-12月-01日    1.1
2019年-12月-02日    1.8
2019年-12月-03日    2.0
2019年-12月-04日    2.0
2019年-12月-05日    1.9
2019年-12月-06日    2.1
2019年-12月-07日    1.7
2019年-12月-08日    2.3
2019年-12月-09日    2.0
2019年-12月-10日    2.3
2019年-12月-11日    1.9
2019年-12月-12日    1.7
2019年-12月-13日    1.4
2019年-12月-14日    1.7
2019年-12月-15日    1.4
2019年-12月-16日    1.7
2019年-12月-17日    1.8
2019年-12月-18日    1.4
2019年-12月-19日    2.0
2019年-12月-20日    2.3
2019年-12月-21日    1.9
2019年-12月-22日    2.2
2019年-12月-23日    1.6
2019年-12月-24日    1.2
2019年-12月-25日    2.3
2019年-12月-26日    2.3
2019年-12月-27日    2.4
2019年-12月-28日    2.7
2019年-12月-29日    2.6
2019年-12月-30日    2.4
2019年-12月-31日    2.1
Name: 2分钟定时风速平均, dtype: float64


In [68]:
#新建dataframe
hour_24=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时']
print('2分钟风速风向：')
df_FN1_fx_point=pd.DataFrame(data=fx1_point,index=index,columns=hour_24)
print(df_FN1_fx_point)

2分钟风速风向：
               21时  22时  23时  24时   1时   2时   3时   4时   5时   6时  ...  11时  \
2019年-12月-01日  PPC  PPC  PPC    E  PPC   NE  NNE  PPC   NE   NE  ...  WSW   
2019年-12月-02日  NNW    N  NNE  NNW    N  NNE  NNE    W  PPC  PPC  ...   SW   
2019年-12月-03日   NE   NE   NE  ENE    E  ENE  ENE  ENE    E    E  ...   SW   
2019年-12月-04日  ENE    E  ENE  ENE  ENE    E    E  ENE    E   NE  ...   SW   
2019年-12月-05日  ENE  ENE  ENE    E  ENE    E    E   NE    E    E  ...   SW   
2019年-12月-06日    E  ENE  ENE  ENE   NE    E    E  ENE    E    E  ...   SW   
2019年-12月-07日  ENE  ENE   NE  ENE    E  ENE  ENE   NE    E    E  ...   SW   
2019年-12月-08日   NE   NE  ENE  ENE  ENE  ENE   NE    E  ENE  ENE  ...  WSW   
2019年-12月-09日  ENE  ENE  ENE  ENE    E  ENE  ENE  ENE   NE   NE  ...   SW   
2019年-12月-10日  ENE  ENE  ENE  ENE    E  ENE  ENE   NE   NE   NE  ...   NW   
2019年-12月-11日   NE  ENE  ENE  ENE  ENE  ESE   NE  ENE   NE  ENE  ...   SW   
2019年-12月-12日  ENE    E  ENE  ENE    E    E  ENE  ESE  ENE   NE  ..

In [69]:
#新建dataframe
hour_24=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时']
print('2分钟定时风速风向：')
df_FN1_fx_angle=pd.DataFrame(data=fx1_angle,index=index,columns=hour_24)
print(df_FN1_fx_angle)

2分钟定时风速风向：
                 21时    22时    23时    24时     1时     2时     3时     4时     5时  \
2019年-12月-01日    0.0    0.0    0.0   90.0    0.0   45.0   22.5    0.0   45.0   
2019年-12月-02日  337.5    0.0   22.5  337.5    0.0   22.5   22.5  270.0    0.0   
2019年-12月-03日   45.0   45.0   45.0   67.5   90.0   67.5   67.5   67.5   90.0   
2019年-12月-04日   67.5   90.0   67.5   67.5   67.5   90.0   90.0   67.5   90.0   
2019年-12月-05日   67.5   67.5   67.5   90.0   67.5   90.0   90.0   45.0   90.0   
2019年-12月-06日   90.0   67.5   67.5   67.5   45.0   90.0   90.0   67.5   90.0   
2019年-12月-07日   67.5   67.5   45.0   67.5   90.0   67.5   67.5   45.0   90.0   
2019年-12月-08日   45.0   45.0   67.5   67.5   67.5   67.5   45.0   90.0   67.5   
2019年-12月-09日   67.5   67.5   67.5   67.5   90.0   67.5   67.5   67.5   45.0   
2019年-12月-10日   67.5   67.5   67.5   67.5   90.0   67.5   67.5   45.0   45.0   
2019年-12月-11日   45.0   67.5   67.5   67.5   67.5  112.5   45.0   67.5   45.0   
2019年-12月-12日   67.5   90.0  

In [70]:
#FN_2 第2段：十分钟分钟
FN_2=FN[fn[0]+1:fn[1]+1]
print(len(FN_2))
print(FN_2)

124
['308003 PPC000 PPC002 051005 069007 030004', '027012 PPC000 037007 039009 094005 089005', '107009 240018 229031 216032 134019 288033', '317020 347018 291018 103012 185006 047012.', '343031 350049 002033 345029 357026 030012', '021017 258012 047005 276004 026009 095011', '079017 208014 242029 323030 269027 299029', '281021 009019 014014 025021 078011 048024.', '047032 056025 047036 066022 086017 057025', '058026 076019 101017 096016 068019 111017', '173009 168008 225021 239033 165019 269043', '289025 314019 001016 063014 062011 054021.', '067022 073019 061032 067018 063016 075018', '071020 060013 080010 041020 069013 102015', '083019 206010 231026 219027 137016 304034', '303016 001018 032024 080010 043017 051023.', '062024 064025 071028 080016 063020 090020', '069016 058015 102014 104014 066009 053007', '142009 224013 229020 218036 218042 219046', '215027 216033 242014 098012 058012 058015.', '081023 070024 071030 065027 048019 077022', '081020 067025 087016 084012 083011 058019', 

In [71]:
#对 FN_1 内的数据进行拼接,生成  fn_1
fn_2=[]  
for i in range(len(FN_2)+1):
    if i != 0:
        if i%4==0:   #能被 4 整除
            fn_2=fn_2+[FN_2[i-4]+' '+FN_2[i-3]+' '+FN_2[i-2]+' '+FN_2[i-1]]
print(len(fn_2))
#print(fn_1)

31


In [72]:
#去掉.和对不同的=结尾进行分割,在 pc_1 上直接进行处理
for i in range(len(fn_2)):
    if fn_2[i][-1] in fn0:
        fn_2[i]=fn_2[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        fn_2[i]=fn_2[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(fn_2)):
    fn_2[i]=fn_2[i].split()
print(len(fn_2))
#print(fn_2)

31


In [73]:
#格式转换
fx=[]  #风向
fs=[]  #风速
for i in range(len(fn_2)):
    for j in range(len(fn_2[i])):
            fx=fx+[fn_2[i][j][:3]]
            fs=fs+[int(fn_2[i][j][3:])/10]

In [74]:
fx_point=[]  #方位
fx_angle=[]  #中心角度
'''
风向按照度数进行划分，划分为16个风向

'''
for i in range(len(fx)):
    if fx[i]=='PPC':
        fx_point=fx_point+['PPC']    #C-PPC 静风，风速小于0.2m/s 
        fx_angle=fx_angle+[0]
    else:
        if int(fx[i])>=0 and int(fx[i])<=11.25:
            fx_point=fx_point+['N']
            fx_angle=fx_angle+[0]
        elif int(fx[i])>=11.26 and int(fx[i])<=33.75:
            fx_point=fx_point+['NNE']
            fx_angle=fx_angle+[22.5]
        elif int(fx[i])>=33.76 and int(fx[i])<=56.25:
            fx_point=fx_point+['NE']
            fx_angle=fx_angle+[45]
        elif int(fx[i])>=56.26 and int(fx[i])<=78.75:
            fx_point=fx_point+['ENE']
            fx_angle=fx_angle+[67.5]
        elif int(fx[i])>=78.76 and int(fx[i])<=101.25:
            fx_point=fx_point+['E']
            fx_angle=fx_angle+[90]
        elif int(fx[i])>=101.26 and int(fx[i])<=123.75:
            fx_point=fx_point+['ESE']
            fx_angle=fx_angle+[112.5]
        if int(fx[i])>=123.76 and int(fx[i])<=146.25:
            fx_point=fx_point+['SE']
            fx_angle=fx_angle+[135]
        elif int(fx[i])>=146.26 and int(fx[i])<=168.75:
            fx_point=fx_point+['SSE']
            fx_angle=fx_angle+[157.5]
        elif int(fx[i])>=168.76 and int(fx[i])<=191.25:
            fx_point=fx_point+['S']
            fx_angle=fx_angle+[180]
        elif int(fx[i])>=191.26 and int(fx[i])<=213.75:
            fx_point=fx_point+['SSW']
            fx_angle=fx_angle+[202.5]
        elif int(fx[i])>=213.76 and int(fx[i])<=236.25:
            fx_point=fx_point+['SW']
            fx_angle=fx_angle+[225]
        elif int(fx[i])>=236.26 and int(fx[i])<=258.75:
            fx_point=fx_point+['WSW']
            fx_angle=fx_angle+[247.5]
        elif int(fx[i])>=258.76 and int(fx[i])<=281.25:
            fx_point=fx_point+['W']
            fx_angle=fx_angle+[270]
        elif int(fx[i])>=281.76 and int(fx[i])<=303.75:
            fx_point=fx_point+['WNW']
            fx_angle=fx_angle+[295.5]
        elif int(fx[i])>=303.76 and int(fx[i])<=326.25:
            fx_point=fx_point+['NW']
            fx_angle=fx_angle+[315]
        elif int(fx[i])>=326.26 and int(fx[i])<=348.75:
            fx_point=fx_point+['NNW']
            fx_angle=fx_angle+[337.5]
        elif int(fx[i])>=348.76 and int(fx[i])<=360:
            fx_point=fx_point+['N']
            fx_angle=fx_angle+[0]

In [75]:
fx2_point=np.array(fx_point).reshape(-1,24) 
print(len(fx2_point))  #方位

31


In [76]:
fx2_angle=np.array(fx_angle).reshape(-1,24) 
print(len(fx2_angle))

31


In [77]:
fs2=np.array(fs).reshape(-1,24) 
print(len(fs2))

31


In [78]:
#新建dataframe
hour_24=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时']
print('十分钟风速风向：')
df_FN2_fs=pd.DataFrame(data=fs2,index=index,columns=hour_24)
print(df_FN2_fs)

十分钟风速风向：
               21时  22时  23时  24时   1时   2时   3时   4时   5时   6时  ...  11时  \
2019年-12月-01日  0.3  0.0  0.2  0.5  0.7  0.4  1.2  0.0  0.7  0.9  ...  3.1   
2019年-12月-02日  3.1  4.9  3.3  2.9  2.6  1.2  1.7  1.2  0.5  0.4  ...  2.9   
2019年-12月-03日  3.2  2.5  3.6  2.2  1.7  2.5  2.6  1.9  1.7  1.6  ...  2.1   
2019年-12月-04日  2.2  1.9  3.2  1.8  1.6  1.8  2.0  1.3  1.0  2.0  ...  2.6   
2019年-12月-05日  2.4  2.5  2.8  1.6  2.0  2.0  1.6  1.5  1.4  1.4  ...  2.0   
2019年-12月-06日  2.3  2.4  3.0  2.7  1.9  2.2  2.0  2.5  1.6  1.2  ...  1.3   
2019年-12月-07日  1.9  2.3  1.4  3.1  2.4  1.1  1.5  1.3  1.8  0.7  ...  1.7   
2019年-12月-08日  1.5  2.0  2.0  2.0  2.3  2.3  2.2  1.9  1.9  2.5  ...  1.8   
2019年-12月-09日  2.1  2.6  2.5  2.8  2.4  2.7  2.6  1.9  0.8  1.9  ...  1.6   
2019年-12月-10日  2.6  2.4  2.3  2.6  2.1  2.0  2.1  1.7  1.2  1.6  ...  1.6   
2019年-12月-11日  2.2  2.1  2.4  2.9  1.8  1.5  1.0  2.5  0.7  1.1  ...  2.9   
2019年-12月-12日  2.3  2.2  2.8  2.6  1.1  1.9  2.2  1.2  1.3  1.2  ..

In [79]:
df_FN2_fs['十分钟定时风速']=df_FN2_fs['21时']+df_FN2_fs['22时']+df_FN2_fs['23时']+df_FN2_fs['24时']+\
            df_FN2_fs['1时']+df_FN2_fs['2时']+df_FN2_fs['3时']+df_FN2_fs['4时']+df_FN2_fs['5时']+\
            df_FN2_fs['6时']+df_FN2_fs['7时']+df_FN2_fs['8时']+df_FN2_fs['9时']+df_FN2_fs['10时']+\
            df_FN2_fs['11时']+df_FN2_fs['12时']+df_FN2_fs['13时']+df_FN2_fs['14时']+df_FN2_fs['15时']+\
            df_FN2_fs['16时']+df_FN2_fs['17时']+df_FN2_fs['18时']+df_FN2_fs['19时']+df_FN2_fs['20时']
print(df_FN2_fs['十分钟定时风速'])

2019年-12月-01日    28.7
2019年-12月-02日    49.4
2019年-12月-03日    51.0
2019年-12月-04日    45.6
2019年-12月-05日    48.7
2019年-12月-06日    49.3
2019年-12月-07日    40.7
2019年-12月-08日    50.9
2019年-12月-09日    47.6
2019年-12月-10日    56.8
2019年-12月-11日    46.7
2019年-12月-12日    41.3
2019年-12月-13日    33.5
2019年-12月-14日    39.5
2019年-12月-15日    32.0
2019年-12月-16日    40.5
2019年-12月-17日    40.4
2019年-12月-18日    33.8
2019年-12月-19日    49.0
2019年-12月-20日    53.3
2019年-12月-21日    46.6
2019年-12月-22日    52.7
2019年-12月-23日    37.4
2019年-12月-24日    26.6
2019年-12月-25日    57.8
2019年-12月-26日    55.0
2019年-12月-27日    58.3
2019年-12月-28日    63.9
2019年-12月-29日    62.1
2019年-12月-30日    54.8
2019年-12月-31日    47.6
Name: 十分钟定时风速, dtype: float64


In [80]:
df_FN2_fs['十分钟定时风速日平均']=round(df_FN2_fs['十分钟定时风速']/24,1)
print(df_FN2_fs['十分钟定时风速日平均'])

2019年-12月-01日    1.2
2019年-12月-02日    2.1
2019年-12月-03日    2.1
2019年-12月-04日    1.9
2019年-12月-05日    2.0
2019年-12月-06日    2.1
2019年-12月-07日    1.7
2019年-12月-08日    2.1
2019年-12月-09日    2.0
2019年-12月-10日    2.4
2019年-12月-11日    1.9
2019年-12月-12日    1.7
2019年-12月-13日    1.4
2019年-12月-14日    1.6
2019年-12月-15日    1.3
2019年-12月-16日    1.7
2019年-12月-17日    1.7
2019年-12月-18日    1.4
2019年-12月-19日    2.0
2019年-12月-20日    2.2
2019年-12月-21日    1.9
2019年-12月-22日    2.2
2019年-12月-23日    1.6
2019年-12月-24日    1.1
2019年-12月-25日    2.4
2019年-12月-26日    2.3
2019年-12月-27日    2.4
2019年-12月-28日    2.7
2019年-12月-29日    2.6
2019年-12月-30日    2.3
2019年-12月-31日    2.0
Name: 十分钟定时风速日平均, dtype: float64


In [81]:
#新建dataframe
hour_24=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时']
print('十分钟定时风速风向：')
df_FN2_fx_point=pd.DataFrame(data=fx2_point,index=index,columns=hour_24)
print(df_FN2_fx_point)

十分钟定时风速风向：
               21时  22时  23时  24时   1时   2时   3时   4时   5时   6时  ...  11时  \
2019年-12月-01日   NW  PPC  PPC   NE  ENE  NNE  NNE  PPC   NE   NE  ...   SW   
2019年-12月-02日  NNW    N    N  NNW    N  NNE  NNE  WSW   NE    W  ...  WSW   
2019年-12月-03日   NE   NE   NE  ENE    E  ENE  ENE  ENE    E    E  ...   SW   
2019年-12月-04日  ENE  ENE  ENE  ENE  ENE  ENE  ENE  ENE    E   NE  ...   SW   
2019年-12月-05日  ENE  ENE  ENE    E  ENE    E  ENE  ENE  ESE  ESE  ...   SW   
2019年-12月-06日    E  ENE  ENE  ENE   NE  ENE    E  ENE    E    E  ...  SSW   
2019年-12月-07日   NE  ENE  ENE  ENE  ENE  ENE    E  ENE    E  ENE  ...   SW   
2019年-12月-08日   NE   NE  ENE  ENE  ENE  ENE   NE  ENE  ENE  ENE  ...   SW   
2019年-12月-09日  ENE  ENE  ENE  ENE    E  ENE  ENE    E  NNE   NE  ...   SW   
2019年-12月-10日  ENE  ENE  ENE    E    E  ENE  ENE   NE   NE   NE  ...   NW   
2019年-12月-11日   NE  ENE  ENE  ENE   NE    E   NE  ENE  NNE   NE  ...   SW   
2019年-12月-12日  ENE    E  ENE  ENE    E    E  ENE    E   NE   NE  

In [82]:
#新建dataframe
hour_24=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时']
print('十分钟定时风速风向：')
df_FN2_fx_angle=pd.DataFrame(data=fx2_angle,index=index,columns=hour_24)
print(df_FN2_fx_angle)

十分钟定时风速风向：
                 21时    22时    23时    24时     1时     2时     3时     4时     5时  \
2019年-12月-01日  315.0    0.0    0.0   45.0   67.5   22.5   22.5    0.0   45.0   
2019年-12月-02日  337.5    0.0    0.0  337.5    0.0   22.5   22.5  247.5   45.0   
2019年-12月-03日   45.0   45.0   45.0   67.5   90.0   67.5   67.5   67.5   90.0   
2019年-12月-04日   67.5   67.5   67.5   67.5   67.5   67.5   67.5   67.5   90.0   
2019年-12月-05日   67.5   67.5   67.5   90.0   67.5   90.0   67.5   67.5  112.5   
2019年-12月-06日   90.0   67.5   67.5   67.5   45.0   67.5   90.0   67.5   90.0   
2019年-12月-07日   45.0   67.5   67.5   67.5   67.5   67.5   90.0   67.5   90.0   
2019年-12月-08日   45.0   45.0   67.5   67.5   67.5   67.5   45.0   67.5   67.5   
2019年-12月-09日   67.5   67.5   67.5   67.5   90.0   67.5   67.5   90.0   22.5   
2019年-12月-10日   67.5   67.5   67.5   90.0   90.0   67.5   67.5   45.0   45.0   
2019年-12月-11日   45.0   67.5   67.5   67.5   45.0   90.0   45.0   67.5   22.5   
2019年-12月-12日   67.5   90.0  

In [83]:
#FN_3 第3组   每天最大极大风及时间
FN_3=FN[fn[1]+1:fn[2]+1]
print(FN_3)

['042263 1326 069227 1317', '059348 2139 107351 2219', '050272 1406 075281 1426', '043306 1355 081306 1318', '047218 1354 065213 1357', '033066 0028 047235 1359', '036057 0024 049057 0023', '052219 1433 068258 1437', '049224 1640 065224 1636', '053264 1216 081281 1446', '039218 1214 054031 1552', '034044 1432 055048 2114', '032243 1057 045207 1313', '049120 1139 087119 1134', '031243 1322 043241 1318', '051309 1201 084351 1358', '051041 1621 074037 1621', '033358 1501 056340 1442', '045235 1153 062249 1146', '049230 1535 074224 1529', '038210 1457 054244 1451', '045218 1306 059235 1504', '041223 1234 053221 1228', '033236 1516 044227 1314', '066307 1300 116334 1712', '051346 2021 092360 2012', '046236 1549 066300 1221', '063230 1334 087224 1332', '059308 1455 086040 1629', '062047 1012 089312 1507', '050237 1441 061034 2020=']


In [84]:
#去掉.和对不同的=结尾进行分割,在 pc_1 上直接进行处理
fn0=['=','.']
for i in range(len(FN_3)):
    if FN_3[i][-1] in fn0:
        FN_3[i]=FN_3[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        FN_3[i]=FN_3[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(FN_3)):
    FN_3[i]=FN_3[i].split()
print(len(FN_3))
#print(FN_3)

31


In [85]:
zd_fx=[]  #日最大,前三位-风速，后三位-风向
zd_fs=[]
zd_time=[]
jd_fx=[]   #日极大,前三位-风速，后三位-风向
jd_fs=[]
jd_time=[]
for i in range(len(FN_3)):
    for j in range(len(FN_3[i])):
        if j==0:
            zd_fx=zd_fx+[FN_3[i][j][3:]] 
            zd_fs=zd_fs+[int(FN_3[i][j][:3])/10]
        elif j==1:
            zd_time=zd_time+[FN_3[i][j][:2]+':'+FN_3[i][j][2:]]
        elif j==2:
            jd_fx=jd_fx+[FN_3[i][j][3:]]
            jd_fs=jd_fs+[int(FN_3[i][j][:3])/10]
        elif j==3:
            jd_time=jd_time+[FN_3[i][j][:2]+':'+FN_3[i][j][2:]]
            

In [86]:
print(zd_fx)  #风向
print(zd_fs)
print(zd_time)

['263', '348', '272', '306', '218', '066', '057', '219', '224', '264', '218', '044', '243', '120', '243', '309', '041', '358', '235', '230', '210', '218', '223', '236', '307', '346', '236', '230', '308', '047', '237']
[4.2, 5.9, 5.0, 4.3, 4.7, 3.3, 3.6, 5.2, 4.9, 5.3, 3.9, 3.4, 3.2, 4.9, 3.1, 5.1, 5.1, 3.3, 4.5, 4.9, 3.8, 4.5, 4.1, 3.3, 6.6, 5.1, 4.6, 6.3, 5.9, 6.2, 5.0]
['13:26', '21:39', '14:06', '13:55', '13:54', '00:28', '00:24', '14:33', '16:40', '12:16', '12:14', '14:32', '10:57', '11:39', '13:22', '12:01', '16:21', '15:01', '11:53', '15:35', '14:57', '13:06', '12:34', '15:16', '13:00', '20:21', '15:49', '13:34', '14:55', '10:12', '14:41']


In [87]:
print(jd_fx)  #风向
print(jd_fs)
print(jd_time)

['227', '351', '281', '306', '213', '235', '057', '258', '224', '281', '031', '048', '207', '119', '241', '351', '037', '340', '249', '224', '244', '235', '221', '227', '334', '360', '300', '224', '040', '312', '034']
[6.9, 10.7, 7.5, 8.1, 6.5, 4.7, 4.9, 6.8, 6.5, 8.1, 5.4, 5.5, 4.5, 8.7, 4.3, 8.4, 7.4, 5.6, 6.2, 7.4, 5.4, 5.9, 5.3, 4.4, 11.6, 9.2, 6.6, 8.7, 8.6, 8.9, 6.1]
['13:17', '22:19', '14:26', '13:18', '13:57', '13:59', '00:23', '14:37', '16:36', '14:46', '15:52', '21:14', '13:13', '11:34', '13:18', '13:58', '16:21', '14:42', '11:46', '15:29', '14:51', '15:04', '12:28', '13:14', '17:12', '20:12', '12:21', '13:32', '16:29', '15:07', '20:20']


In [88]:
'''
对最大风向进行处理
'''
zd_fx_point=[]
zd_fx_angle=[]
for i in range(len(zd_fx)):
    if zd_fx[i]=='PPC':
        zd_fx_point=zd_fx_point+['PPC']    #C-PPC 静风，风速小于0.2m/s 
        zd_fx_angle=zd_fx_angle+[0]
    else:
        if int(zd_fx[i])>=0 and int(zd_fx[i])<=11.25:
            zd_fx_point=zd_fx_point+['N']
            zd_fx_angle=zd_fx_angle+[0]
        elif int(zd_fx[i])>=11.26 and int(zd_fx[i])<=33.75:
            zd_fx_point=zd_fx_point+['NNE']
            zd_fx_angle=zd_fx_angle+[22.5]
        elif int(zd_fx[i])>=33.76 and int(zd_fx[i])<=56.25:
            zd_fx_point=zd_fx_point+['NE']
            zd_fx_angle=zd_fx_angle+[45]
        elif int(zd_fx[i])>=56.26 and int(zd_fx[i])<=78.75:
            zd_fx_point=zd_fx_point+['ENE']
            zd_fx_angle=zd_fx_angle+[67.5]
        elif int(zd_fx[i])>=78.76 and int(zd_fx[i])<=101.25:
            zd_fx_point=zd_fx_point+['E']
            zd_fx_angle=zd_fx_angle+[90]
        elif int(zd_fx[i])>=101.26 and int(zd_fx[i])<=123.75:
            zd_fx_point=zd_fx_point+['ESE']
            zd_fx_angle=zd_fx_angle+[112.5]
        if int(zd_fx[i])>=123.76 and int(zd_fx[i])<=146.25:
            zd_fx_point=zd_fx_point+['SE']
            zd_fx_angle=zd_fx_angle+[135]
        elif int(zd_fx[i])>=146.26 and int(zd_fx[i])<=168.75:
            zd_fx_point=zd_fx_point+['SSE']
            zd_fx_angle=zd_fx_angle+[157.5]
        elif int(zd_fx[i])>=168.76 and int(zd_fx[i])<=191.25:
            zd_fx_point=zd_fx_point+['S']
            zd_fx_angle=zd_fx_angle+[180]
        elif int(zd_fx[i])>=191.26 and int(zd_fx[i])<=213.75:
            zd_fx_point=zd_fx_point+['SSW']
            zd_fx_angle=zd_fx_angle+[202.5]
        elif int(zd_fx[i])>=213.76 and int(zd_fx[i])<=236.25:
            zd_fx_point=zd_fx_point+['SW']
            zd_fx_angle=zd_fx_angle+[225]
        elif int(zd_fx[i])>=236.26 and int(zd_fx[i])<=258.75:
            zd_fx_point=zd_fx_point+['WSW']
            zd_fx_angle=zd_fx_angle+[247.5]
        elif int(zd_fx[i])>=258.76 and int(zd_fx[i])<=281.25:
            zd_fx_point=zd_fx_point+['W']
            zd_fx_angle=zd_fx_angle+[270]
        elif int(zd_fx[i])>=281.76 and int(zd_fx[i])<=303.75:
            zd_fx_point=zd_fx_point+['WNW']
            zd_fx_angle=zd_fx_angle+[295.5]
        elif int(zd_fx[i])>=303.76 and int(zd_fx[i])<=326.25:
            zd_fx_point=zd_fx_point+['NW']
            zd_fx_angle=zd_fx_angle+[315]
        elif int(zd_fx[i])>=326.26 and int(zd_fx[i])<=348.75:
            zd_fx_point=zd_fx_point+['NNW']
            zd_fx_angle=zd_fx_angle+[337.5]
        elif int(zd_fx[i])>=348.76 and int(zd_fx[i])<=360:
            zd_fx_point=zd_fx_point+['N']
            zd_fx_angle=zd_fx_angle+[0]

In [89]:
'''
对极大风向进行处理
'''
jd_fx_point=[]
jd_fx_angle=[]
for i in range(len(jd_fx)):
    if jd_fx[i]=='PPC':
        jd_fx_point=jd_fx_point+['PPC']    #C-PPC 静风，风速小于0.2m/s 
        jd_fx_angle=jd_fx_angle+[0]
    else:
        if int(jd_fx[i])>=0 and int(jd_fx[i])<=11.25:
            jd_fx_point=jd_fx_point+['N']
            jd_fx_angle=jd_fx_angle+[0]
        elif int(jd_fx[i])>=11.26 and int(jd_fx[i])<=33.75:
            jd_fx_point=jd_fx_point+['NNE']
            jd_fx_angle=jd_fx_angle+[22.5]
        elif int(jd_fx[i])>=33.76 and int(jd_fx[i])<=56.25:
            jd_fx_point=jd_fx_point+['NE']
            jd_fx_angle=jd_fx_angle+[45]
        elif int(jd_fx[i])>=56.26 and int(jd_fx[i])<=78.75:
            jd_fx_point=jd_fx_point+['ENE']
            jd_fx_angle=jd_fx_angle+[67.5]
        elif int(jd_fx[i])>=78.76 and int(jd_fx[i])<=101.25:
            jd_fx_point=jd_fx_point+['E']
            jd_fx_angle=jd_fx_angle+[90]
        elif int(jd_fx[i])>=101.26 and int(jd_fx[i])<=123.75:
            jd_fx_point=jd_fx_point+['ESE']
            jd_fx_angle=jd_fx_angle+[112.5]
        if int(jd_fx[i])>=123.76 and int(jd_fx[i])<=146.25:
            jd_fx_point=jd_fx_point+['SE']
            jd_fx_angle=jd_fx_angle+[135]
        elif int(jd_fx[i])>=146.26 and int(jd_fx[i])<=168.75:
            jd_fx_point=jd_fx_point+['SSE']
            jd_fx_angle=jd_fx_angle+[157.5]
        elif int(jd_fx[i])>=168.76 and int(jd_fx[i])<=191.25:
            jd_fx_point=jd_fx_point+['S']
            jd_fx_angle=jd_fx_angle+[180]
        elif int(jd_fx[i])>=191.26 and int(jd_fx[i])<=213.75:
            jd_fx_point=jd_fx_point+['SSW']
            jd_fx_angle=jd_fx_angle+[202.5]
        elif int(jd_fx[i])>=213.76 and int(jd_fx[i])<=236.25:
            jd_fx_point=jd_fx_point+['SW']
            jd_fx_angle=jd_fx_angle+[225]
        elif int(jd_fx[i])>=236.26 and int(jd_fx[i])<=258.75:
            jd_fx_point=jd_fx_point+['WSW']
            jd_fx_angle=jd_fx_angle+[247.5]
        elif int(jd_fx[i])>=258.76 and int(jd_fx[i])<=281.25:
            jd_fx_point=jd_fx_point+['W']
            jd_fx_angle=jd_fx_angle+[270]
        elif int(jd_fx[i])>=281.76 and int(jd_fx[i])<=303.75:
            jd_fx_point=jd_fx_point+['WNW']
            jd_fx_angle=jd_fx_angle+[295.5]
        elif int(jd_fx[i])>=303.76 and int(jd_fx[i])<=326.25:
            jd_fx_point=jd_fx_point+['NW']
            jd_fx_angle=jd_fx_angle+[315]
        elif int(jd_fx[i])>=326.26 and int(jd_fx[i])<=348.75:
            jd_fx_point=jd_fx_point+['NNW']
            jd_fx_angle=jd_fx_angle+[337.5]
        elif int(jd_fx[i])>=348.76 and int(jd_fx[i])<=360:
            jd_fx_point=jd_fx_point+['N']
            jd_fx_angle=jd_fx_angle+[0]

In [90]:
#新建dataframe
#hour_8=['日最大风速','日最大风速风向','日最大风速风向中心角度','时间',
#       '日极大风速','日极大风速风向','日极大风速风向中心角度','时间'
#       ]
data_ZJ={'日最大风速':zd_fs , '日最大风速风向':zd_fx_point , '日最大风速风向中心角度':zd_fx_angle, '最大时间':zd_time ,
       '日极大风速':jd_fs , '日极大风速风向':jd_fx_point , '日极大风速风向中心角度':zd_fx_angle , '极大时间':jd_time}
print('最大极大风：')
df_FN_ZJ=pd.DataFrame(data=data_ZJ,index=index)
print(df_FN_ZJ.columns)
print(df_FN_ZJ)

最大极大风：
Index(['日最大风速', '日最大风速风向', '日最大风速风向中心角度', '最大时间', '日极大风速', '日极大风速风向',
       '日极大风速风向中心角度', '极大时间'],
      dtype='object')
               日最大风速 日最大风速风向  日最大风速风向中心角度   最大时间  日极大风速 日极大风速风向  日极大风速风向中心角度  \
2019年-12月-01日    4.2       W        270.0  13:26    6.9      SW        270.0   
2019年-12月-02日    5.9     NNW        337.5  21:39   10.7       N        337.5   
2019年-12月-03日    5.0       W        270.0  14:06    7.5       W        270.0   
2019年-12月-04日    4.3      NW        315.0  13:55    8.1      NW        315.0   
2019年-12月-05日    4.7      SW        225.0  13:54    6.5     SSW        225.0   
2019年-12月-06日    3.3     ENE         67.5  00:28    4.7      SW         67.5   
2019年-12月-07日    3.6     ENE         67.5  00:24    4.9     ENE         67.5   
2019年-12月-08日    5.2      SW        225.0  14:33    6.8     WSW        225.0   
2019年-12月-09日    4.9      SW        225.0  16:40    6.5      SW        225.0   
2019年-12月-10日    5.3       W        270.0  12:16    8.1       W       

In [91]:
'''
D:浅层地温
B:X=B。
    0cm段每天24次定时和自动观测日最高、最低值及出现时间共28组，分为2个记录， 
    第1个记录（21～08时）为12组，第2个记录（09～20时和最高值及出现时间、最低值及出现时间）为16组；
    5、10、15、20、40cm段，每段每天24次定时值共24组，分为2个记录，每个记录12组。
单位：0.1摄氏度
'''
DB=SJ[SJ.index('DB'):SJ.index('KB')]
print(len(DB))
db=[]
for i in range(len(DB)):
    if DB[i][-1]=='=':  # 若字符串最后一位为=
        db=db+[i]
print(db)
#print(DB)

373
[62, 124, 186, 248, 310, 372]


In [92]:
#DB_1 :24小时值和最小、最低值及时间
DB_1 =DB[1:db[0]+1]
print(len(DB_1))
#print(DB_1)

62


In [93]:
#对 DB_1 内的数据进行拼接,生成db_1
db_1=[]  
for i in range(len(DB_1)+1):
    if i != 0:
        if i%2==0:   #能被2整除
            db_1=db_1+[DB_1[i-2]+' '+DB_1[i-1]]
print(len(db_1))
#print(db_1)

31


In [94]:
#去掉.和对不同的=结尾进行分割,在 pc_1 上直接进行处理
db0=['=','.']
for i in range(len(db_1)):
    if db_1[i][-1] in pc0:
        db_1[i]=db_1[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        db_1[i]=db_1[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(db_1)):
    db_1[i]=db_1[i].split()
print(len(db_1))
#print(db_1)

31


In [95]:
#格式转换
for i in range(len(db_1)):
    for j in range(len(db_1[i])):
        if j == 25 or j==27:
            db_1[i][j]='%.2f' %(int(db_1[i][j])/100)
            db_1[i][j]=db_1[i][j].replace('.',':')
        else: 
            if db_1[i][j][0]=='-':  #如果是负数
                if db_1[i][j][1]=='0':
                    db_1[i][j]=db_1[i][j][0]+db_1[i][j][2]+'.'+db_1[i][j][3]
                else:
                    db_1[i][j]=db_1[i][j][0]+db_1[i][j][1]+db_1[i][j][2]+'.'+db_1[i][j][3]
                        
            else:  #如果是正数
                db_1[i][j]=int(db_1[i][j])/10
#print(db_1)                   

In [96]:
#新建dataframe
hour_28=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时',
         '最大','时间','最小','时间',]
print('浅层地温：')
df_DB_1=pd.DataFrame(data=db_1,index=index,columns=hour_28)
print(df_DB_1)

浅层地温：
                21时   22时   23时   24时    1时     2时     3时     4时     5时  \
2019年-12月-01日  -0.3  -0.3  -0.4  -0.8  -1.0   -1.4   -1.9   -1.7   -2.1   
2019年-12月-02日  -2.0  -2.3  -2.8  -3.4  -3.9   -4.5   -4.9   -5.4   -5.7   
2019年-12月-03日  -4.2  -4.7  -5.2  -5.5  -5.9   -6.3   -6.6   -6.9   -7.1   
2019年-12月-04日  -4.0  -4.5  -4.9  -5.4  -5.8   -6.1   -6.4   -6.7   -7.0   
2019年-12月-05日  -3.8  -4.3  -4.7  -5.2  -5.5   -5.9   -6.3   -6.6   -6.9   
2019年-12月-06日  -4.0  -4.4  -4.8  -5.2  -5.6   -5.9   -6.3   -6.5   -6.8   
2019年-12月-07日  -3.9  -4.3  -4.8  -5.0  -5.3   -5.8   -6.0   -6.3   -6.4   
2019年-12月-08日  -3.0  -2.8  -3.5  -3.8  -4.1   -4.4   -4.7   -5.1   -5.3   
2019年-12月-09日  -2.9  -3.2  -3.6  -3.8  -4.2   -4.4   -4.6   -5.0   -5.5   
2019年-12月-10日  -2.4  -2.8  -3.2  -3.5  -3.8   -4.2   -4.4   -4.7   -4.8   
2019年-12月-11日  -3.2  -3.6  -3.9  -4.2  -4.6   -5.0   -5.4   -5.6   -6.1   
2019年-12月-12日  -3.4  -3.7  -4.0  -4.3  -4.9   -5.1   -5.3   -5.5   -5.7   
2019年-12月-13日  -0.3

In [97]:
#DB_5 :5cm地温
DB_5 =DB[db[0]+1:db[1]+1]
print(len(DB_5))
#print(DB_5)

62


In [98]:
#对 DB_5 内的数据进行拼接,生成db_5
db_5=[]
for i in range(len(DB_5)+1):
    if i != 0:
        if i%2==0:   #能被2整除
            db_5=db_5+[DB_5[i-2]+' '+DB_5[i-1]]
print(len(db_5))
#print(db_5)

31


In [99]:
#去掉.和对不同的=结尾进行分割,在 pc_1 上直接进行处理
for i in range(len(db_5)):
    if db_5[i][-1] in db0:
        db_5[i]=db_5[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        db_5[i]=db_5[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(db_5)):
    db_5[i]=db_5[i].split()
print(len(db_5))
#print(db_5)

31


In [100]:
#格式转换
for i in range(len(db_5)):
    for j in range(len(db_5[i])):
        if db_5[i][j][0]=='-':  #如果是负数
            if db_5[i][j][1]=='0':
                db_5[i][j]=float(db_5[i][j][0]+db_5[i][j][2]+'.'+db_5[i][j][3])
            else:
                db_5[i][j]=float(db_5[i][j][0]+db_5[i][j][1]+db_5[i][j][2]+'.'+db_5[i][j][3])
                        
        else:  #如果是正数
            db_5[i][j]=float(int(db_5[i][j])/10)
#print(db_5) 

In [101]:
#新建dataframe
hour_24=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时']
print('5cm地温：')
df_DB_5=pd.DataFrame(data=db_5,index=index,columns=hour_24)
print(df_DB_5)

5cm地温：
               21时  22时  23时  24时   1时   2时   3时   4时   5时   6时  ...  11时  \
2019年-12月-01日  1.2  1.0  0.9  0.8  0.6  0.5  0.4  0.2  0.1  0.0  ... -0.2   
2019年-12月-02日  0.6  0.4  0.2  0.1 -0.1 -0.3 -0.7 -1.1 -1.5 -1.9  ... -0.7   
2019年-12月-03日 -0.1 -0.2 -0.5 -1.0 -1.4 -1.7 -2.0 -2.2 -2.5 -2.7  ... -0.8   
2019年-12月-04日 -0.2 -0.2 -0.4 -0.9 -1.3 -1.6 -1.8 -2.1 -2.3 -2.6  ... -0.8   
2019年-12月-05日 -0.2 -0.2 -0.3 -0.7 -1.1 -1.4 -1.6 -1.9 -2.2 -2.4  ... -0.7   
2019年-12月-06日 -0.2 -0.2 -0.4 -0.8 -1.1 -1.4 -1.7 -2.0 -2.2 -2.4  ... -0.8   
2019年-12月-07日 -0.2 -0.2 -0.3 -0.7 -1.1 -1.4 -1.6 -1.9 -2.1 -2.3  ... -0.7   
2019年-12月-08日 -0.2 -0.2 -0.3 -0.3 -0.5 -0.8 -1.0 -1.2 -1.4 -1.6  ... -0.6   
2019年-12月-09日 -0.2 -0.2 -0.2 -0.3 -0.4 -0.7 -0.9 -1.1 -1.3 -1.5  ... -0.5   
2019年-12月-10日  0.0 -0.1 -0.1 -0.2 -0.2 -0.3 -0.4 -0.6 -0.9 -1.0  ... -0.4   
2019年-12月-11日  0.1  0.0 -0.1 -0.2 -0.2 -0.4 -0.7 -1.0 -1.3 -1.5  ... -0.5   
2019年-12月-12日 -0.1 -0.1 -0.2 -0.2 -0.4 -0.7 -1.0 -1.2 -1.4 -1.5  ... 

In [102]:
df_DB_5['5cm地温日总和']=df_DB_5['21时']+df_DB_5['22时']+df_DB_5['23时']+df_DB_5['24时']+\
            df_DB_5['1时']+df_DB_5['2时']+df_DB_5['3时']+df_DB_5['4时']+df_DB_5['5时']+\
            df_DB_5['6时']+df_DB_5['7时']+df_DB_5['8时']+df_DB_5['9时']+df_DB_5['10时']+\
            df_DB_5['11时']+df_DB_5['12时']+df_DB_5['13时']+df_DB_5['14时']+df_DB_5['15时']+\
            df_DB_5['16时']+df_DB_5['17时']+df_DB_5['18时']+df_DB_5['19时']+df_DB_5['20时']
print(df_DB_5['5cm地温日总和'])

2019年-12月-01日    34.9
2019年-12月-02日    -6.4
2019年-12月-03日   -19.4
2019年-12月-04日   -18.0
2019年-12月-05日   -16.9
2019年-12月-06日   -17.8
2019年-12月-07日   -15.5
2019年-12月-08日    -6.7
2019年-12月-09日     7.5
2019年-12月-10日    10.0
2019年-12月-11日    -2.1
2019年-12月-12日   -15.6
2019年-12月-13日     6.9
2019年-12月-14日    -4.2
2019年-12月-15日     5.2
2019年-12月-16日    52.0
2019年-12月-17日    28.1
2019年-12月-18日    26.6
2019年-12月-19日   -11.0
2019年-12月-20日   -19.2
2019年-12月-21日   -15.8
2019年-12月-22日   -16.6
2019年-12月-23日   -26.5
2019年-12月-24日    -6.7
2019年-12月-25日     2.3
2019年-12月-26日   -17.2
2019年-12月-27日   -28.1
2019年-12月-28日   -20.2
2019年-12月-29日   -16.4
2019年-12月-30日   -22.4
2019年-12月-31日   -66.7
Name: 5cm地温日总和, dtype: float64


In [103]:
df_DB_5['5cm地温日平均']=round(df_DB_5['5cm地温日总和']/24,1)
print(df_DB_5['5cm地温日平均'])

2019年-12月-01日    1.5
2019年-12月-02日   -0.3
2019年-12月-03日   -0.8
2019年-12月-04日   -0.7
2019年-12月-05日   -0.7
2019年-12月-06日   -0.7
2019年-12月-07日   -0.6
2019年-12月-08日   -0.3
2019年-12月-09日    0.3
2019年-12月-10日    0.4
2019年-12月-11日   -0.1
2019年-12月-12日   -0.7
2019年-12月-13日    0.3
2019年-12月-14日   -0.2
2019年-12月-15日    0.2
2019年-12月-16日    2.2
2019年-12月-17日    1.2
2019年-12月-18日    1.1
2019年-12月-19日   -0.5
2019年-12月-20日   -0.8
2019年-12月-21日   -0.7
2019年-12月-22日   -0.7
2019年-12月-23日   -1.1
2019年-12月-24日   -0.3
2019年-12月-25日    0.1
2019年-12月-26日   -0.7
2019年-12月-27日   -1.2
2019年-12月-28日   -0.8
2019年-12月-29日   -0.7
2019年-12月-30日   -0.9
2019年-12月-31日   -2.8
Name: 5cm地温日平均, dtype: float64


In [104]:
#DB_10 :10cm地温
DB_10 =DB[db[1]+1:db[2]+1]
print(len(DB_10))
#print(DB_10)

62


In [105]:
#对 DB_10 内的数据进行拼接,生成db_10
db_10=[]
for i in range(len(DB_10)+1):
    if i != 0:
        if i%2==0:   #能被2整除
            db_10=db_10+[DB_10[i-2]+' '+DB_10[i-1]]
print(len(db_10))
#print(db_10)

31


In [106]:
#去掉.和对不同的=结尾进行分割,在 db_10 上直接进行处理
for i in range(len(db_10)):
    if db_10[i][-1] in db0:
        db_10[i]=db_10[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        db_10[i]=db_10[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(db_10)):
    db_10[i]=db_10[i].split()
print(len(db_10))
#print(db_10)

31


In [107]:
#格式转换
for i in range(len(db_10)):
    for j in range(len(db_10[i])):
        if db_10[i][j][0]=='-':  #如果是负数
            if db_10[i][j][1]=='0':
                db_10[i][j]=float(db_10[i][j][0]+db_10[i][j][2]+'.'+db_10[i][j][3])
            else:
                db_10[i][j]=float(db_10[i][j][0]+db_10[i][j][1]+db_10[i][j][2]+'.'+db_10[i][j][3])

        else:  #如果是正数
            db_10[i][j]=float(int(db_10[i][j])/10)
#print(db_10) 

In [108]:
#新建dataframe
print('10cm地温：')
df_DB_10=pd.DataFrame(data=db_10,index=index,columns=hour_24)
print(df_DB_10)

10cm地温：
               21时  22时  23时  24时   1时   2时   3时   4时   5时   6时  ...  11时  \
2019年-12月-01日  2.2  2.1  1.9  1.8  1.7  1.6  1.4  1.3  1.2  1.1  ...  0.7   
2019年-12月-02日  2.0  1.7  1.5  1.3  1.1  1.0  0.8  0.6  0.5  0.4  ...  0.0   
2019年-12月-03日  0.5  0.5  0.4  0.3  0.2  0.2  0.1  0.0  0.0 -0.1  ... -0.5   
2019年-12月-04日 -0.2 -0.1 -0.1 -0.1 -0.1 -0.1 -0.1 -0.1 -0.1 -0.2  ... -0.5   
2019年-12月-05日 -0.2 -0.2 -0.2 -0.2 -0.2 -0.1 -0.1 -0.1 -0.2 -0.2  ... -0.5   
2019年-12月-06日 -0.3 -0.2 -0.2 -0.2 -0.2 -0.2 -0.2 -0.2 -0.2 -0.3  ... -0.6   
2019年-12月-07日 -0.3 -0.3 -0.3 -0.3 -0.2 -0.2 -0.2 -0.2 -0.2 -0.3  ... -0.6   
2019年-12月-08日 -0.3 -0.3 -0.3 -0.3 -0.3 -0.3 -0.2 -0.2 -0.2 -0.2  ... -0.4   
2019年-12月-09日 -0.2 -0.2 -0.2 -0.2 -0.2 -0.2 -0.2 -0.2 -0.2 -0.2  ... -0.3   
2019年-12月-10日  0.4  0.3  0.2  0.2  0.1  0.1  0.1  0.0  0.0  0.0  ... -0.1   
2019年-12月-11日  0.7  0.5  0.4  0.3  0.3  0.2  0.1  0.1  0.0  0.0  ... -0.2   
2019年-12月-12日  0.3  0.2  0.2  0.2  0.1  0.1  0.0  0.0 -0.1 -0.1  ...

In [109]:
df_DB_10['10cm地温日总和']=df_DB_10['21时']+df_DB_10['22时']+df_DB_10['23时']+df_DB_10['24时']+\
            df_DB_10['1时']+df_DB_10['2时']+df_DB_10['3时']+df_DB_10['4时']+df_DB_10['5时']+\
            df_DB_10['6时']+df_DB_10['7时']+df_DB_10['8时']+df_DB_10['9时']+df_DB_10['10时']+\
            df_DB_10['11时']+df_DB_10['12时']+df_DB_10['13时']+df_DB_10['14时']+df_DB_10['15时']+\
            df_DB_10['16时']+df_DB_10['17时']+df_DB_10['18时']+df_DB_10['19时']+df_DB_10['20时']
print(df_DB_10['10cm地温日总和'])

2019年-12月-01日    46.1
2019年-12月-02日    14.3
2019年-12月-03日    -2.6
2019年-12月-04日    -6.3
2019年-12月-05日    -7.0
2019年-12月-06日    -8.4
2019年-12月-07日    -8.9
2019年-12月-08日    -6.7
2019年-12月-09日     1.6
2019年-12月-10日     9.8
2019年-12月-11日     3.9
2019年-12月-12日    -1.8
2019年-12月-13日     4.3
2019年-12月-14日     4.1
2019年-12月-15日     9.7
2019年-12月-16日    44.2
2019年-12月-17日    33.5
2019年-12月-18日    29.3
2019年-12月-19日     4.1
2019年-12月-20日    -6.4
2019年-12月-21日    -9.2
2019年-12月-22日   -10.4
2019年-12月-23日   -11.8
2019年-12月-24日    -8.9
2019年-12月-25日    -6.8
2019年-12月-26日    -6.9
2019年-12月-27日   -16.0
2019年-12月-28日   -16.1
2019年-12月-29日   -12.7
2019年-12月-30日   -13.6
2019年-12月-31日   -36.0
Name: 10cm地温日总和, dtype: float64


In [110]:
df_DB_10['10cm地温日平均']=round(df_DB_10['10cm地温日总和']/24,1)
print(df_DB_10['10cm地温日平均'])

2019年-12月-01日    1.9
2019年-12月-02日    0.6
2019年-12月-03日   -0.1
2019年-12月-04日   -0.3
2019年-12月-05日   -0.3
2019年-12月-06日   -0.4
2019年-12月-07日   -0.4
2019年-12月-08日   -0.3
2019年-12月-09日    0.1
2019年-12月-10日    0.4
2019年-12月-11日    0.2
2019年-12月-12日   -0.1
2019年-12月-13日    0.2
2019年-12月-14日    0.2
2019年-12月-15日    0.4
2019年-12月-16日    1.8
2019年-12月-17日    1.4
2019年-12月-18日    1.2
2019年-12月-19日    0.2
2019年-12月-20日   -0.3
2019年-12月-21日   -0.4
2019年-12月-22日   -0.4
2019年-12月-23日   -0.5
2019年-12月-24日   -0.4
2019年-12月-25日   -0.3
2019年-12月-26日   -0.3
2019年-12月-27日   -0.7
2019年-12月-28日   -0.7
2019年-12月-29日   -0.5
2019年-12月-30日   -0.6
2019年-12月-31日   -1.5
Name: 10cm地温日平均, dtype: float64


In [111]:
#DB_15 :15cm地温
DB_15 =DB[db[2]+1:db[3]+1]
print(len(DB_15))
#print(DB_15)

62


In [112]:
#对 DB_5 内的数据进行拼接,生成db_5
db_15=[]
for i in range(len(DB_15)+1):
    if i != 0:
        if i%2==0:   #能被2整除
            db_15=db_15+[DB_15[i-2]+' '+DB_15[i-1]]
print(len(db_15))
#print(db_15)

31


In [113]:
#去掉.和对不同的=结尾进行分割,在 pc_1 上直接进行处理
for i in range(len(db_15)):
    if db_15[i][-1] in db0:
        db_15[i]=db_15[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        db_15[i]=db_15[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(db_15)):
    db_15[i]=db_15[i].split()
print(len(db_15))
#print(db_15)

31


In [114]:
#格式转换
for i in range(len(db_15)):
    for j in range(len(db_15[i])):
        if db_15[i][j][0]=='-':  #如果是负数
            if db_15[i][j][1]=='0':
                db_15[i][j]=float(db_15[i][j][0]+db_15[i][j][2]+'.'+db_15[i][j][3])
            else:
                db_15[i][j]=float(db_15[i][j][0]+db_15[i][j][1]+db_15[i][j][2]+'.'+db_15[i][j][3])

        else:  #如果是正数
            db_15[i][j]=float(int(db_15[i][j])/10)
#print(db_15) 

In [115]:
#新建dataframe
print('15cm地温：')
df_DB_15=pd.DataFrame(data=db_15,index=index,columns=hour_24)
print(df_DB_15)

15cm地温：
               21时  22时  23时  24时   1时   2时   3时   4时   5时   6时  ...  11时  \
2019年-12月-01日  3.4  3.2  3.1  3.0  2.9  2.8  2.7  2.6  2.5  2.4  ...  2.0   
2019年-12月-02日  3.2  3.0  2.8  2.7  2.5  2.4  2.2  2.1  2.0  1.9  ...  1.4   
2019年-12月-03日  1.6  1.5  1.5  1.5  1.4  1.4  1.3  1.3  1.2  1.2  ...  0.8   
2019年-12月-04日  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  ...  0.6   
2019年-12月-05日  0.7  0.7  0.7  0.8  0.8  0.8  0.8  0.8  0.8  0.8  ...  0.6   
2019年-12月-06日  0.6  0.6  0.7  0.7  0.7  0.7  0.7  0.7  0.7  0.7  ...  0.5   
2019年-12月-07日  0.5  0.5  0.6  0.6  0.6  0.6  0.6  0.6  0.6  0.6  ...  0.4   
2019年-12月-08日  0.5  0.5  0.5  0.5  0.5  0.5  0.6  0.6  0.6  0.6  ...  0.5   
2019年-12月-09日  0.6  0.6  0.6  0.6  0.6  0.6  0.6  0.6  0.6  0.6  ...  0.5   
2019年-12月-10日  1.1  1.0  1.0  0.9  0.9  0.9  0.8  0.8  0.8  0.8  ...  0.7   
2019年-12月-11日  1.4  1.3  1.2  1.2  1.1  1.0  1.0  0.9  0.9  0.9  ...  0.7   
2019年-12月-12日  1.0  1.0  0.9  0.9  0.9  0.9  0.8  0.8  0.8  0.7  ...

In [116]:
df_DB_15['15cm地温日总和']=df_DB_15['21时']+df_DB_15['22时']+df_DB_15['23时']+df_DB_15['24时']+\
            df_DB_15['1时']+df_DB_15['2时']+df_DB_15['3时']+df_DB_15['4时']+df_DB_15['5时']+\
            df_DB_15['6时']+df_DB_15['7时']+df_DB_15['8时']+df_DB_15['9时']+df_DB_15['10时']+\
            df_DB_15['11时']+df_DB_15['12时']+df_DB_15['13时']+df_DB_15['14时']+df_DB_15['15时']+\
            df_DB_15['16时']+df_DB_15['17时']+df_DB_15['18时']+df_DB_15['19时']+df_DB_15['20时']
print(df_DB_15['15cm地温日总和'])

2019年-12月-01日    67.7
2019年-12月-02日    45.5
2019年-12月-03日    25.9
2019年-12月-04日    18.4
2019年-12月-05日    16.0
2019年-12月-06日    13.9
2019年-12月-07日    12.2
2019年-12月-08日    13.0
2019年-12月-09日    17.3
2019年-12月-10日    23.8
2019年-12月-11日    22.1
2019年-12月-12日    17.4
2019年-12月-13日    18.5
2019年-12月-14日    21.5
2019年-12月-15日    24.3
2019年-12月-16日    47.5
2019年-12月-17日    48.0
2019年-12月-18日    42.8
2019年-12月-19日    27.1
2019年-12月-20日    13.5
2019年-12月-21日     9.3
2019年-12月-22日     7.5
2019年-12月-23日     5.3
2019年-12月-24日     5.7
2019年-12月-25日     7.6
2019年-12月-26日     8.3
2019年-12月-27日     3.2
2019年-12月-28日     0.7
2019年-12月-29日     0.3
2019年-12月-30日     0.7
2019年-12月-31日    -7.3
Name: 15cm地温日总和, dtype: float64


In [117]:
df_DB_15['15cm地温日平均']=round(df_DB_15['15cm地温日总和']/24,1)
print(df_DB_15['15cm地温日平均'])

2019年-12月-01日    2.8
2019年-12月-02日    1.9
2019年-12月-03日    1.1
2019年-12月-04日    0.8
2019年-12月-05日    0.7
2019年-12月-06日    0.6
2019年-12月-07日    0.5
2019年-12月-08日    0.5
2019年-12月-09日    0.7
2019年-12月-10日    1.0
2019年-12月-11日    0.9
2019年-12月-12日    0.7
2019年-12月-13日    0.8
2019年-12月-14日    0.9
2019年-12月-15日    1.0
2019年-12月-16日    2.0
2019年-12月-17日    2.0
2019年-12月-18日    1.8
2019年-12月-19日    1.1
2019年-12月-20日    0.6
2019年-12月-21日    0.4
2019年-12月-22日    0.3
2019年-12月-23日    0.2
2019年-12月-24日    0.2
2019年-12月-25日    0.3
2019年-12月-26日    0.3
2019年-12月-27日    0.1
2019年-12月-28日    0.0
2019年-12月-29日    0.0
2019年-12月-30日    0.0
2019年-12月-31日   -0.3
Name: 15cm地温日平均, dtype: float64


In [118]:
'''
S:日照时数
2:
    X=2。每天各时（3～21）日照时数共18组及日照总时数1组。
    X=A。每天各时（1～24）日照时数共24组及日出时间、日落时间、日照总时数各1组。
单位：0.1小时
'''

'\nS:日照时数\n2:\n    X=2。每天各时（3～21）日照时数共18组及日照总时数1组。\n    X=A。每天各时（1～24）日照时数共24组及日出时间、日落时间、日照总时数各1组。\n单位：0.1小时\n'

In [119]:
SA=SJ[SJ.index('SA')+1:SJ.index('BA')]

In [120]:
#去掉.和对不同的=结尾进行分割,在 pc_1 上直接进行处理
s20=['=','.']
for i in range(len(SA)):
    if SA[i][-1] in s20:
        SA[i]=SA[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        SA[i]=SA[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(SA)):
    SA[i]=SA[i].split()
print(len(SA))
#print(SA)

31


In [121]:
#格式转换
for i in range(len(SA)):
    for j in range(len(SA[i])):
        if j==24 or j==25:
            SA[i][j]='%.2f' %(int(SA[i][j])/100)
            SA[i][j]=SA[i][j].replace('.',':')
        else:
            if SA[i][j]!='NN':
                SA[i][j]=float(int(SA[i][j])/10)
            else:
                SA[i][j]='NN'

In [122]:
print(SA)

[['NN', 'NN', 'NN', 'NN', 'NN', 'NN', 0.0, 0.3, 1.0, 0.7, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3, 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', '6:55', '16:43', 8.3], ['NN', 'NN', 'NN', 'NN', 'NN', 'NN', 0.0, 0.7, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.4, 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', '6:56', '16:43', 9.1], ['NN', 'NN', 'NN', 'NN', 'NN', 'NN', 0.0, 0.7, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.4, 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', '6:57', '16:43', 9.1], ['NN', 'NN', 'NN', 'NN', 'NN', 'NN', 0.0, 0.7, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.4, 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', '6:58', '16:42', 9.1], ['NN', 'NN', 'NN', 'NN', 'NN', 'NN', 0.0, 0.7, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3, 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', '6:58', '16:42', 9.0], ['NN', 'NN', 'NN', 'NN', 'NN', 'NN', 0.0, 0.4, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2, 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', '6:59', '16:42', 8.6], ['NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 0.3, 1.0, 1.0,

In [123]:
#新建dataframe
hour_27=['1时','2时','3时','4时','5时','6时', '7时','8时','9时','10时','11时','12时','13时','14时','15时',
         '16时','17时','18时','19时','20时','21时','22时','23时','24时','日出时间','日落时间','日照总时数']
print('日照时数：')
df_SA=pd.DataFrame(data=SA,index=index,columns=hour_27)
print(df_SA)

日照时数：
               1时  2时  3时  4时  5时  6时  7时   8时   9时  10时  ...  18时  19时  20时  \
2019年-12月-01日  NN  NN  NN  NN  NN  NN   0  0.3  1.0  0.7  ...   NN   NN   NN   
2019年-12月-02日  NN  NN  NN  NN  NN  NN   0  0.7  1.0  1.0  ...   NN   NN   NN   
2019年-12月-03日  NN  NN  NN  NN  NN  NN   0  0.7  1.0  1.0  ...   NN   NN   NN   
2019年-12月-04日  NN  NN  NN  NN  NN  NN   0  0.7  1.0  1.0  ...   NN   NN   NN   
2019年-12月-05日  NN  NN  NN  NN  NN  NN   0  0.7  1.0  1.0  ...   NN   NN   NN   
2019年-12月-06日  NN  NN  NN  NN  NN  NN   0  0.4  1.0  1.0  ...   NN   NN   NN   
2019年-12月-07日  NN  NN  NN  NN  NN  NN  NN  0.3  1.0  1.0  ...   NN   NN   NN   
2019年-12月-08日  NN  NN  NN  NN  NN  NN  NN  0.0  0.9  1.0  ...   NN   NN   NN   
2019年-12月-09日  NN  NN  NN  NN  NN  NN  NN  0.5  1.0  1.0  ...   NN   NN   NN   
2019年-12月-10日  NN  NN  NN  NN  NN  NN  NN  0.5  1.0  1.0  ...   NN   NN   NN   
2019年-12月-11日  NN  NN  NN  NN  NN  NN  NN  0.6  1.0  1.0  ...   NN   NN   NN   
2019年-12月-12日  NN  NN  NN  NN  NN 

In [124]:
filename='A'+station[0]+'-'+station[-2]+'-'+station[-1]+'.'+'xlsx'
print(filename)

A53859-2019-12.xlsx


In [125]:
'''
保存，将多个保存在一个excel中

'''
from openpyxl import load_workbook
 
writer = pd.ExcelWriter(filename)

df_station.to_excel(excel_writer=writer,sheet_name="站点信息")

df_PC_1.to_excel(excel_writer=writer,sheet_name="本站气压")
df_PC_2.to_excel(excel_writer=writer,sheet_name="海平面气压")

df_TB.to_excel(excel_writer=writer,sheet_name="气温")

df_UB.to_excel(excel_writer=writer,sheet_name="相对湿度")

df_R6_1.to_excel(excel_writer=writer,sheet_name="定时降水量")
df_R6_2.to_excel(excel_writer=writer,sheet_name="降水量")

df_FN1_fs.to_excel(excel_writer=writer,sheet_name="2分钟定时风速") 
df_FN1_fx_point.to_excel(excel_writer=writer,sheet_name="2分钟定时风速风向方位")  
df_FN1_fx_angle.to_excel(excel_writer=writer,sheet_name="2分钟定时风速风向中心角度")  
df_FN2_fs.to_excel(excel_writer=writer,sheet_name="10分钟定时风速") 
df_FN2_fx_point.to_excel(excel_writer=writer,sheet_name="10分钟定时风速风向方位")  
df_FN2_fx_angle.to_excel(excel_writer=writer,sheet_name="10分钟定时风速风向中心角度")  
df_FN_ZJ.to_excel(excel_writer=writer,sheet_name="日最大极大风速风向") 

df_DB_1.to_excel(excel_writer=writer,sheet_name="浅层地温")
df_DB_5.to_excel(excel_writer=writer,sheet_name="5cm地温")
df_DB_10.to_excel(excel_writer=writer,sheet_name="10cm地温")
df_DB_15.to_excel(excel_writer=writer,sheet_name="15cm地温")

#df_S2.to_excel(excel_writer=writer,sheet_name="日照时数")
df_SA.to_excel(excel_writer=writer,sheet_name="日照时数")

writer.save()
writer.close()
print('导入excel成功')

导入excel成功
